## Predicting coronary artery disease (CAD) diagnosis from patient physiological parameters via K-nearest neighbour classification tools.

### Introduction

Coronary artery disease (CAD) is a form of heart disease derived from the constriction of coronary arteries and the narrowing of blood vessels, which can lead to restricted blood flow through the heart (CDC, 2021). Over time, this restriction can weaken the heart and lead to heart failure. 

There are several factors that influence the risk of developing CAD. For example, relationships exist between a patient’s heart health and their age, resting blood pressure, cholesterol level, and maximum heart rate, to name a few.  Since it is estimated that 47% of all Americans have at least one of these risk factors for the disease (CDC, 2021), this classification model aims to answer the following research question: **Can the diagnosis of coronary artery disease (diagnosed when blood vessels narrow >50%) be accurately predicted based on patient physiological parameters?**

The dataset used to answer this question is a multivariate dataset derived from Detrano, R. et al, a paper published in 1988 used to apply a probability algorithm to the diagnosis of coronary artery disease and donated to University of California Irvine’s Machine Learning Repository.




### Methods

After cleaning and wrangling the data, the data is split into training and testing data sets. Classification with K-nearest neighbours will be used to compute the response variable (which is the presence of heart disease, indicated in the code as “healthy” or “sick”) based on previous diagnoses in the training data set that align most closely with the predictor variables given in the new observation. The predictor variables are age, resting blood pressure (bp), cholesterol, and maximum heart rate. These variables are chosen because they are most relevant to the focus of the study as numerical physiological parameters. 

Conversely, several variables are excluded that are present in the original data set, including sex, chest pain type, fasting blood sugar, resting electrocardiographic results, exercise induced angina, the slope of the peak exercise ST segment, and the category of defect. This is because they are all presented as categorical values, so summary statistics and calculations for the straight line distance would not be operational in a K-nearest neighbour classification using the tools learned in class. The ST segment depression induced by the exercise variable and number of blood vessels (numerical) are excluded as it is not clear what they represent in the dataset and is not directly relevant since measures induced by exercise are not the focus of the current study.

The classifier is built by developing a workflow that combines our model specification (to specify that this is a classification model using the “kknn” R package, to set number of neighbors we want our classifier to predict from, and to indicate the weight each neighboring point should have) and a recipe (to preprocess the training set and standardize it, preventing our classifier from placing importance on variables whose observations have larger values). The training set is split into a training set and a validation set to optimize the value of K by cross-validation. 

Subsequently, the model is fit to the training set. Once the classifier object is determined, it is used to predict class labels for the test set and to examine the accuracy and confusion matrix, evaluating the classifier’s performance.

From this model, a series of scatterplots visualize the relationships between two parameters that our classifier accounts for (e.g. cholesterol levels and age), with the diagnosis or absence of being coloured using a color blind-friendly palette. Significant patterns will be described from the comparison of these variables. A bar graph is also developed to visually show the confusion matrix.



**Importing libraries**

In [5]:
library(tidyverse)
library(repr)
library(dplyr)
library(tidymodels)
library(infer)
library(ggplot2)

install.packages("kknn")
install.packages("stringr")
install.packages("cowplot")
install.packages("GGally")

library(stringr)
library(kknn)
library(cowplot)
library(GGally)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.4     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

**Demonstrate that the dataset can be read from the web into R**

In [6]:
# downloading data to fiel_path
url <- "https://archive.ics.uci.edu/static/public/45/heart+disease.zip"
temp_file <- tempfile(fileext = ".zip")
download.file(url, destfile = temp_file, mode = "wb")
unzip(temp_file, exdir = "GroupProject/heart_data")
file_path <- "GroupProject/heart_data/cleve.mod"


# reading data into heart_data
heart_data <- read_table(file_path, skip = 20)


── Column specification ────────────────────────────────────────────────────────
cols(
  `63.0` = col_double(),
  male = col_character(),
  angina = col_character(),
  `145.0` = col_double(),
  `233.0` = col_double(),
  true = col_character(),
  hyp = col_character(),
  `150.0` = col_double(),
  fal = col_character(),
  `2.3` = col_double(),
  down = col_character(),
  `0.0` = col_character(),
  fix = col_character(),
  buff = col_character(),
  H = col_character()
)



**Clean and wrangle your data into a tidy format**

In [7]:
# assigning  names to each column
names <- c("age","sex","chest_pain_type","resting_bp",
                    "cholesterol","fasting_blood_sugar", "resting_ecg",
                    "max_heart_rate","exercise_induced_angina","oldpeak",
                    "slope","number_of_vessels_colored","thal","healthy")
colnames(heart_data) <- names

# selecting useful columns (see below for justification of exclusion of oldpeak and slope)
heart_data <- heart_data|> 
    select(age,sex,resting_bp,cholesterol,max_heart_rate, healthy)

head(heart_data)

age,sex,resting_bp,cholesterol,max_heart_rate,healthy
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
67,male,160,286,108,sick
67,male,120,229,129,sick
37,male,130,250,187,buff
41,fem,130,204,172,buff
56,male,120,236,178,buff
62,fem,140,268,160,sick


In [8]:
** table 1 **

ERROR: Error in parse(text = x, srcfile = src): <text>:1:1: unexpected '^'
1: **
    ^


**Spliting data**

In [ ]:
# splitting data into training and testing
set.seed(2004)

#Turning healthy and sex into factors

heart_data |>
  distinct(healthy)
heart_data <- heart_data |>
  mutate(healthy = as_factor(healthy)) |>
  mutate(healthy = fct_recode(healthy, "sick" = "sick", "healthy" = "buff"))

heart_data <-heart_data|>
  mutate(sex=as_factor(sex))|>
  mutate(sex = fct_recode(sex, "male" = "male", "female" = "fem"))

head(heart_data)


heart_disease_split <- initial_split(heart_data,prop = 0.75, strata = healthy)
heart_disease_train <- training(heart_disease_split)
heart_disease_test <- testing(heart_disease_split)

In [ ]:
**Table 2**

In [ ]:
# I plan to use Trestbps, Cholesteral and Max_Heart_Rate as predictors
# None of the rows have missing data since we already addressed this issue when importing the data
# Datasets below shows the count for healhty and sick observations and mean of each of our predictors 


heart_disease_info <- heart_disease_train |>
    mutate(observation_count = n()) |>
    mutate(mean_resting_bp = mean(resting_bp)) |>
    mutate(mean_cholesterol = mean(cholesterol)) |>
    mutate(mean_max_heart_rate = mean(max_heart_rate)) |>
    mutate(mean_age = mean(age)) |>
    #mutate(mean_number_of_blood_vesels = mean(number_of_vessels_colored)) |>
    select(observation_count,mean_age,mean_resting_bp,mean_cholesterol,mean_max_heart_rate) |>
    distinct()

sick_data <- heart_disease_train |>
    filter(healthy == "sick") |>
    mutate(observation_count = n()) |>
    mutate(sick_mean_resting_bp = mean(resting_bp)) |>
    mutate(sick_mean_cholesterol = mean(cholesterol)) |>
    mutate(sick_mean_max_heart_rate = mean(max_heart_rate)) |>
    mutate(sick_mean_age = mean(age)) |>
    #mutate(sick_mean_number_of_blood_vesels = mean(number_of_vessels_colored)) |>
    select(observation_count,sick_mean_age,sick_mean_resting_bp,sick_mean_cholesterol,sick_mean_max_heart_rate) |>
    distinct()

healthy_data <- heart_disease_train |>
    filter(healthy == "buff") |>
    mutate(observation_count = n()) |>
    mutate(healthy_mean_resting_bp = mean(resting_bp)) |>
    mutate(healthy_mean_cholesterol = mean(cholesterol)) |>
    mutate(healthy_mean_max_heart_rate = mean(max_heart_rate)) |>
    mutate(healthy_mean_age = mean(age)) |>
    #mutate(healthy_mean_number_of_blood_vesels = mean(healthy_number_of_vessels_colored)) |>
    select(observation_count,healthy_mean_age,healthy_mean_resting_bp,healthy_mean_cholesterol,healthy_mean_max_heart_rate) |>
    distinct()

heart_disease_info
sick_data
healthy_data

In [ ]:
**table 3**

In [ ]:
#summarizing the data by getting number of sick and healthy
healthy_proportions <- heart_disease_train |>
                      group_by(healthy) |>
                      summarize(n = n()) |>
                      mutate(percent = 100*n/nrow(heart_disease_train))
healthy_proportions

In [ ]:
**table 4**

In [ ]:
#Classifying Part A: choosing the K value
set.seed(1920)
heart_recipe <- recipe(healthy ~ age+resting_bp+cholesterol+max_heart_rate, data = heart_disease_train) |>
  step_scale(all_predictors()) |>
  step_center(all_predictors())

knn_spec <- nearest_neighbor(weight_func = "rectangular", 
                             neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("classification")

heart_vfold <- vfold_cv(heart_disease_train, v = 10, strata = healthy)
k_vals <- tibble(neighbors = seq(from = 1, to = 100, by = 1))

knn_results <- workflow() |>
  add_recipe(heart_recipe) |>
  add_model(knn_spec) |>
  tune_grid(resamples = heart_vfold, grid = k_vals) |>
  collect_metrics() 

accuracies <- knn_results |>
  filter(.metric == "accuracy")

accuracy_vs_k <- ggplot(accuracies, aes(x = neighbors, y = mean)) +
  geom_point() +
  geom_line() +
  labs(x = "Neighbors", y = "Accuracy Estimate", caption = "figure 1") + 
  theme(text = element_text(size = 12))

accuracy_vs_k

**Choosing our K**: 
Looking at the plot above, we can see that setting the K=19 will give us the highest accuracy which is at about 68%. 
Therefore we choose K to be 19. 

In [ ]:
#Classifying the data
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 19) |>
  set_engine("kknn") |>
  set_mode("classification")

knn_fit <- workflow() |>
  add_recipe(heart_recipe) |>
  add_model(knn_spec) |>
  fit(data = heart_disease_train)
knn_fit


heart_test_predictions <- predict(knn_fit, heart_disease_test) |>
  bind_cols(heart_disease_test)

heart_test_predictions

heart_test_predictions |>
  metrics(truth = healthy, estimate = .pred_class) |>
  filter(.metric == "accuracy")


confusion <- heart_test_predictions |>
             conf_mat(truth = healthy, estimate = .pred_class)
confusion

In [ ]:
#Calculating the accuracy, precision, and recall in order to better understand the errors most important in our application. 

Accuracy<- (17+34)/(17+7+18+34)
Accuracy
#67%

Precision<-17/(17+7)
Precision
#71%

Recall<-17/(17+18)
Recall 
#49%

In [ ]:
**table 5**

In [ ]:
#Generating a bar graph to show how accurate our classifier was compared to the actual test observations
bar_plot <- heart_test_predictions |>
    ggplot(aes(x = healthy,fill = .pred_class)) +
    geom_bar(position = "dodge", color = "black") +
    labs(title = "Assessing the Accuracy of CAD Diagnosis Predictions Based on Actual Observations", 
         x = "Actual Observations", 
         y = "Count",
         fill = "CAD Diagnosis",
         caption = "figure 2")

bar_plot

In [ ]:
#Generating the scatterplots to compare each predictor with each other
options(repr.plot.height = 3, repr.plot.width = 4)

age_cholesterol_plot <- heart_test_predictions |>
    ggplot(aes(x = cholesterol, y = age, color = healthy)) +
    geom_point(alpha = 0.6) +
    labs(x = "Cholesterol Level (mg/dL)", y = "Age (years)", color = "CAD Diagnosis", caption = "figure 3") +
    ggtitle("Relationship Between Blood Cholesterol Levels (mg/dL) and Patient Age (years) in CAD Predictions")

age_resting_bp_plot <- heart_test_predictions |>
    ggplot(aes(x = resting_bp, y = age, color = healthy)) +
    geom_point(alpha = 0.6) +
    labs(x = "Resting Blood Pressure (mmHg)", y = "Age (years)", color = "CAD Diagnosis", caption = "figure 4")+
    ggtitle("Relationship Between Resting Blood Pressure (mmHg) and Patient Age (years) in CAD Predictions")

age_max_heart_rate_plot <- heart_test_predictions |>
    ggplot(aes(x = max_heart_rate, y = age, color = healthy)) +
    geom_point(alpha = 0.6) +
    labs(x = "Max Heart Rate (BPM)", y = "Age (years)", color = "CAD Diagnosis", caption = "figure 5") +
    ggtitle("Relationship Between Maximum  Heart Rate (BPM) and Age (years) of the Patient in CAD Predictions")

resting_bp_cholesterol_plot <- heart_test_predictions |>
    ggplot(aes(x = resting_bp, y = cholesterol, color = healthy)) +
    geom_point(alpha = 0.6) +
    labs(x = "Resting Blood Pressure (mmHg)", y = "Cholesterol Level (mg/dL)", color = "CAD Diagnosis", caption = "figure 6")+
    ggtitle("Relationship Between Resting Blood Pressure (mmHg) and Blood Cholesterol Levels (mg/dL) in CAD Predictions")

max_heart_rate_cholesterol_plot <- heart_test_predictions |>
    ggplot(aes(x = max_heart_rate, y = cholesterol, color = healthy)) +
    geom_point(alpha = 0.6) +
    labs(x = "Max Heart Rate (BPM)", y = "Cholesterol Level (md/dL)", color = "CAD Diagnosis", caption = "figure 7")+
    ggtitle("Relationship Between Maximum Heart Rate (BPM) and Blood Cholesterol Levels (mg/dL) in CAD Predictions")

resting_bp_max_heart_rate_plot <- heart_test_predictions |>
    ggplot(aes(x = max_heart_rate, y = resting_bp, color = healthy)) +
    geom_point(alpha = 0.6) +
    labs(x = "Max Heart Rate (BPM)", y = str_wrap("Resting Blood Pressure (mmHg)", width = 20), color = "CAD Diagnosis", caption = "figure 8")+
    ggtitle("Relationship Between Maximum Heart Rate (BPM) and Resting Blood Pressure (mmHg) in CAD Predictions")

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 7)

scatter_plot_panel <- plot_grid(age_cholesterol_plot, 
                                age_max_heart_rate_plot, 
                                age_resting_bp_plot,
                                max_heart_rate_cholesterol_plot, 
                                resting_bp_max_heart_rate_plot, 
                                resting_bp_cholesterol_plot,ncol = 2, nrow = 3)

scatter_plot_panel

### Discussion:

None of the scatterplots show obvious relationships between any of the predictor variables and whether the individual has CAD (sick) or is healthy. In Figure 5, most of the data falls between 100 to 140 mmHg and between roughly 40 to 70 years of age. While it is possible that two groups have formed in the data, in other words within the same blood pressure range, sick diagnoses are predicted at higher ages and healthy diagnoses are predicted at lower ranges, determining whether the formation of these groups is significant (and whether they are present from the outset) would require more analysis. Regardless, because the diagnoses for each phenotype are made irrespective of blood pressure, this suggests that blood pressure is not strongly correlated with age in determining the presence or absence of CAD.

In Figure 3, most data points fall roughly within 40 to 70 years of age and between 200 to 300 mg/dL of serum cholesterol in the blood. However, within this region, there are not clear subgroups for sick or healthy individuals. Similarly, in Figure 4, most of the data falls within ages 40 to 70 and between heart rates of 100 to 200 BPM. Two groups appear to have formed; there are a greater proportion of “sick” observations at higher ages and lower maximum heart rates, and a greater proportion of “healthy” observations at lower ages and higher maximum heart rates. However, the formation of these groups is weak. Drawing more definite inferences about the correlation between these variables and the presence of heart disease would necessitate further analysis to determine the significance of these results. If determined to be significant, it may suggest a positive correlation between age and maximum heart rate that could be used to help predict heart disease.

The distribution of data in Figure 6 shows heart rates between 100 and 175 BPM and cholesterol levels ranging mainly from 175 to 350 mg/dL. No distinction can be made about healthy and sick individuals based on the comparison of these two parameters alone. Similar conclusions can be made about Figure 8, in which most of the resting blood pressure data falls between 100 to 160 mmHg, and cholesterol levels between 175 to 350 mg/dL.

The last scatterplot, Figure 7, shows all of the resting blood pressure data clustering within 100 to 160 mmHg, and maximum heart rates from 100 to 200 BPM. Although more healthy observations tend to be clustered at higher maximum heart rates and more sick observations at lower maximum heart rates, the relative differences between these two are minimal. More observations would likely be needed to determine if it is statistically significant.

Overall, the scatterplots suggest that while age and maximum heart rate may show small degrees of positive correlation with CAD diagnosis relative to the other physiological parameters used in this analysis, the extent of this relationship is minimal and more research is needed to determine whether these results are significant. External research suggests that these predictors are correlated with a CAD diagnosis (CDC, 2021). It is possible that the relationships between these variables in the human body and CAD are more complex and overlapping, making them hard to distinguish in scatterplots. Further casual and mechanistic data analysis questions would be beneficial to uncover these relationships.

The confusion matrix, as well as Figure 2 demonstrate the correct and incorrect “healthy” and “sick” predictions for observations in the testing set. The accuracy of the classifier is 67%, as it correctly predicted 51 out of 76 heart disease cases, which superficially suggests that it made a handful of mistakes in its ability to predict whether patients had heart disease. The model’s precision is 71%, meaning that out of all of the “sick” predictions it made, it correctly identified that 17 patients out of 24 were actually sick. In practice, even though this suggests that the model is not entirely correct, this is not the worst outcome. It is better for healthy patients to receive further attention and medical analysis before contracting CAD than not at all, although it may put a strain on healthcare resources. However, the recall of the classifier is 49% indicating that the model incorrectly evaluated more than half of the actual patients (51%) with coronary artery disease in the test as “healthy”. Misclassifying a sick patient with heart disease is a dire, possibly fatal mistake. 

Overall, the classifier is not necessarily the best model for identifying sick individuals due to its high rates of misclassification. In particular, the model would have been much better had it had higher recall as it is more important in this case. This goal may come at the expense of higher precision, but as iterated throughout this analysis, in fatal circumstances, it is better to overpredict than to underpredict. This idea also poses interesting concerns about using models to diagnose diseases with rapid and severe implications for the patient, and highlights the importance of human input in diagnostic healthcare.

### Conclusion, Future Questions, & Further Directions

In conclusion, this classification model aims to answer whether the diagnosis of CAD could be accurately predicted based on patient physiological parameters. It is found that none of the predicting parameters analyzed in the study are a strong predictor of CAD, despite variables like age and maximum heart rate showing very weak positive correlation. However, the cumulative effect of having multiple risk factors is unknown and may indicate a significant risk of CAD and it indicates that it is still worth watching out for these variables. These findings are crucial as they provide insight as to what factors could be used by physicians to diagnose CAD before it is too late for treatment. Specifically, the findings show that there is no one factor that physicians can use for diagnosis, and that a multitude of factors and their connection need to be analyzed. This can also be used to provide information to the general public about physiological parameters they need to observe about themselves in order to prevent the development of CAD.

 Interesting questions arise from the relative absence of an effect between any of the two parameters chosen and a CAD diagnosis. Firstly, what mechanisms explain why age, resting blood pressure, cholesterol levels, and maximum heart rate are not valid single predictors of cardiac risk? One possibility from the literature indicates that diet plays a large role. If this is the case, is it possible that any dietary components may be successful single predictors? Secondly, is it possible that diversity in patient background (e.g. sex, ethnicity, socioeconomic status, geographical location) contributed too many uncontrolled variables? If so, how might this contribute to physiological differences between patients? 

These questions provide a basis for further work using similar methods outlined above. Environmental and lifestyle factors (other than exercise), such as number of calories consumed per day or number of hours slept per night could be useful as additional predictors of CAD. Execution may require the synthesis of data from multiple data sets or conducting new studies altogether. 

### References

(1) Centres for Disease Control and Prevention. (2021, July 19). Coronary Artery Disease: Causes, Diagnosis & Prevention. Centres for Disease Control and Prevention; U.S. Department of Health & Human Services. https://www.cdc.gov/heartdisease/coronary_ad.html

(2) Spencer, R., Thabtah, F., Abdelhamid, N., & Thompson, M. (2020). Exploring feature selection and classification methods for predicting heart disease. DIGITAL HEALTH, 6, 205520762091477. https://doi.org/10.1177/2055207620914777

(3) Janosti, A. (1988, June 30). UCI Machine Learning Repository. Archive.ics.uci.edu. https://archive.ics.uci.edu/dataset/45/heart+disease